In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
import pandas as pd
from sklearn.utils import shuffle, resample
from afqinsight.augmentation import jitter, time_warp, scaling
import tempfile

In [2]:
afq_dataset = AFQDataset.from_files(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md", "dki_mk"],
    index_col="subject_id",
    target_cols=["age", "dl_qc_score", "scan_site_id"],
    label_encode_cols=["scan_site_id"]
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1865
(1865, 7200)
(1865, 3)


In [5]:
full_dataset = list(afq_dataset.as_tensorflow_dataset().as_numpy_iterator())

2022-05-06 21:06:43.588415: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.concatenate([xx[0][None] for xx in full_dataset], 0)
y = np.array([yy[1][0] for yy in full_dataset])
qc = np.array([yy[1][1] for yy in full_dataset])
site = np.array([yy[1][2] for yy in full_dataset])

In [7]:
X = X[qc>0]
y = y[qc>0]
site = site[qc>0]

In [8]:
n_epochs = 1000

# EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    mode="min",
    patience=100
)

# ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=20,
    verbose=1,
)

In [9]:
def augment_this(X, y, rounds=2): 
    new_X = X[:]
    new_y = y[:]
    for f in range(rounds): 
        aug_X = np.zeros_like(X)
        # Do each channel separately:
        for channel in range(aug_X.shape[-1]):
            this_X = X[..., channel][..., np.newaxis]
            this_X = jitter(this_X, sigma=np.mean(this_X)/25)
            this_X = scaling(this_X, sigma=np.mean(this_X)/25)
            this_X = time_warp(this_X, sigma=np.mean(this_X)/25)
            aug_X[..., channel] = this_X[...,0]
        new_X = np.concatenate([new_X, aug_X])
        new_y = np.concatenate([new_y, y])
    return new_X, new_y 

In [10]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def single_cross_site(model_name, name_str, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train, X_test1, y_train, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    _, X_test2, _, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    _, X_test3, _, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    
    # Augment
    X_train, y_train = augment_this(X_train, y_train, rounds=4)
    X_train, y_train = shuffle(X_train, y_train)
    
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt_filepath = tempfile.NamedTemporaryFile().name + '.h5'
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(name_str + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [site_1]*12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

In [11]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def double_cross_site(model_name, name_str, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    X_train3, X_test3, y_train3, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train1 = np.concatenate([imputer.fit_transform(X_train1[..., ii])[:, :, None] for ii in range(X_train1.shape[-1])], -1)
    X_train2 = np.concatenate([imputer.fit_transform(X_train2[..., ii])[:, :, None] for ii in range(X_train2.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    # size down evenly
    sample = 202
    sample1 = resample(X_train1, y_train1, n_samples=sample, replace=False)
    sample2 = resample(X_train2, y_train2, n_samples=sample, replace=False)
    X_train = np.concatenate((sample1[0], sample2[0]), axis=0)
    y_train = np.concatenate((sample1[1], sample2[1]), axis=0)
    # shuffle
    X_train, y_train = shuffle(X_train, y_train)
    # Augment
    X_train, y_train = augment_this(X_train, y_train, rounds=4)
    X_train, y_train = shuffle(X_train, y_train)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt_filepath = tempfile.NamedTemporaryFile().name + '.h5'
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(name_str + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [f'{site_1}, {site_2}'] * 12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

### cnn_resnet

#### single-cross-site

In [12]:
single0, history_single0 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 3, 4, X, y)
single3, history_single3 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 3, 0, 4, X, y)
single4, history_single4 = single_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 4, 0, 3, X, y)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 100, 64)      36928       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 64)      0           ['batch_normalization[0][0]']

Epoch 1/1000
19/19 [==============================] - ETA: 0s - loss: 30.2540 - mean_squared_error: 30.2540 - rmse: 5.5004 - mean_absolute_error: 4.1301
Epoch 00001: val_loss improved from inf to 1980443264.00000, saving model to /var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/tmpdukpwzch.h5
19/19 [==============================] - 17s 546ms/step - loss: 30.2540 - mean_squared_error: 30.2540 - rmse: 5.5004 - mean_absolute_error: 4.1301 - val_loss: 1980443264.0000 - val_mean_squared_error: 1980443264.0000 - val_rmse: 44502.1719 - val_mean_absolute_error: 44378.2500 - lr: 0.0100
Epoch 2/1000
19/19 [==============================] - ETA: 0s - loss: 5.7493 - mean_squared_error: 5.7493 - rmse: 2.3978 - mean_absolute_error: 1.8839
Epoch 00002: val_loss improved from 1980443264.00000 to 4003088.00000, saving model to /var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/tmpdukpwzch.h5
19/19 [==============================] - 10s 517ms/step - loss: 5.7493 - mean_squared_error: 5.7493 - rmse: 2.397

Epoch 17/1000
19/19 [==============================] - ETA: 0s - loss: 0.8871 - mean_squared_error: 0.8871 - rmse: 0.9419 - mean_absolute_error: 0.7449
Epoch 00017: val_loss did not improve from 5.37569
19/19 [==============================] - 13s 664ms/step - loss: 0.8871 - mean_squared_error: 0.8871 - rmse: 0.9419 - mean_absolute_error: 0.7449 - val_loss: 5.6921 - val_mean_squared_error: 5.6921 - val_rmse: 2.3858 - val_mean_absolute_error: 1.8661 - lr: 0.0100
Epoch 18/1000
19/19 [==============================] - ETA: 0s - loss: 0.9909 - mean_squared_error: 0.9909 - rmse: 0.9954 - mean_absolute_error: 0.7649
Epoch 00018: val_loss did not improve from 5.37569
19/19 [==============================] - 13s 675ms/step - loss: 0.9909 - mean_squared_error: 0.9909 - rmse: 0.9954 - mean_absolute_error: 0.7649 - val_loss: 8.3194 - val_mean_squared_error: 8.3194 - val_rmse: 2.8843 - val_mean_absolute_error: 2.3071 - lr: 0.0100
Epoch 19/1000
19/19 [==============================] - ETA: 0s - los

Epoch 34/1000
19/19 [==============================] - ETA: 0s - loss: 0.6526 - mean_squared_error: 0.6526 - rmse: 0.8078 - mean_absolute_error: 0.6094
Epoch 00034: val_loss did not improve from 3.09254
19/19 [==============================] - 10s 538ms/step - loss: 0.6526 - mean_squared_error: 0.6526 - rmse: 0.8078 - mean_absolute_error: 0.6094 - val_loss: 6.2184 - val_mean_squared_error: 6.2184 - val_rmse: 2.4937 - val_mean_absolute_error: 1.9449 - lr: 0.0100
Epoch 35/1000
19/19 [==============================] - ETA: 0s - loss: 0.5692 - mean_squared_error: 0.5692 - rmse: 0.7545 - mean_absolute_error: 0.5858
Epoch 00035: val_loss did not improve from 3.09254
19/19 [==============================] - 10s 539ms/step - loss: 0.5692 - mean_squared_error: 0.5692 - rmse: 0.7545 - mean_absolute_error: 0.5858 - val_loss: 4.0526 - val_mean_squared_error: 4.0526 - val_rmse: 2.0131 - val_mean_absolute_error: 1.6233 - lr: 0.0100
Epoch 36/1000
19/19 [==============================] - ETA: 0s - los

19/19 [==============================] - ETA: 0s - loss: 0.4892 - mean_squared_error: 0.4892 - rmse: 0.6994 - mean_absolute_error: 0.5342
Epoch 00051: val_loss did not improve from 2.42708
19/19 [==============================] - 12s 641ms/step - loss: 0.4892 - mean_squared_error: 0.4892 - rmse: 0.6994 - mean_absolute_error: 0.5342 - val_loss: 6.3320 - val_mean_squared_error: 6.3320 - val_rmse: 2.5163 - val_mean_absolute_error: 2.0012 - lr: 0.0100
Epoch 52/1000
19/19 [==============================] - ETA: 0s - loss: 0.5083 - mean_squared_error: 0.5083 - rmse: 0.7129 - mean_absolute_error: 0.5571
Epoch 00052: val_loss did not improve from 2.42708
19/19 [==============================] - 12s 662ms/step - loss: 0.5083 - mean_squared_error: 0.5083 - rmse: 0.7129 - mean_absolute_error: 0.5571 - val_loss: 6.2564 - val_mean_squared_error: 6.2564 - val_rmse: 2.5013 - val_mean_absolute_error: 2.0106 - lr: 0.0100
Epoch 53/1000
19/19 [==============================] - ETA: 0s - loss: 0.3275 - me

...

Epoch 335/1000
7/7 [==============================] - ETA: 0s - loss: 0.0352 - mean_squared_error: 0.0352 - rmse: 0.1876 - mean_absolute_error: 0.1526
Epoch 00335: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 421ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - rmse: 0.1876 - mean_absolute_error: 0.1526 - val_loss: 0.4371 - val_mean_squared_error: 0.4371 - val_rmse: 0.6611 - val_mean_absolute_error: 0.5025 - lr: 7.8125e-05
Epoch 336/1000
7/7 [==============================] - ETA: 0s - loss: 0.0741 - mean_squared_error: 0.0741 - rmse: 0.2722 - mean_absolute_error: 0.2187
Epoch 00336: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 479ms/step - loss: 0.0741 - mean_squared_error: 0.0741 - rmse: 0.2722 - mean_absolute_error: 0.2187 - val_loss: 0.4417 - val_mean_squared_error: 0.4417 - val_rmse: 0.6646 - val_mean_absolute_error: 0.5023 - lr: 7.8125e-05
Epoch 337/1000
7/7 [==============================] - ETA: 0s - loss

Epoch 353/1000
7/7 [==============================] - ETA: 0s - loss: 0.0867 - mean_squared_error: 0.0867 - rmse: 0.2945 - mean_absolute_error: 0.2131
Epoch 00353: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 428ms/step - loss: 0.0867 - mean_squared_error: 0.0867 - rmse: 0.2945 - mean_absolute_error: 0.2131 - val_loss: 0.4345 - val_mean_squared_error: 0.4345 - val_rmse: 0.6592 - val_mean_absolute_error: 0.5057 - lr: 3.9062e-05
Epoch 354/1000
7/7 [==============================] - ETA: 0s - loss: 0.1299 - mean_squared_error: 0.1299 - rmse: 0.3604 - mean_absolute_error: 0.2767
Epoch 00354: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 445ms/step - loss: 0.1299 - mean_squared_error: 0.1299 - rmse: 0.3604 - mean_absolute_error: 0.2767 - val_loss: 0.4311 - val_mean_squared_error: 0.4311 - val_rmse: 0.6566 - val_mean_absolute_error: 0.5023 - lr: 3.9062e-05
Epoch 355/1000
7/7 [==============================] - ETA: 0s - loss

Epoch 371/1000
7/7 [==============================] - ETA: 0s - loss: 0.0777 - mean_squared_error: 0.0777 - rmse: 0.2787 - mean_absolute_error: 0.2212
Epoch 00371: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 425ms/step - loss: 0.0777 - mean_squared_error: 0.0777 - rmse: 0.2787 - mean_absolute_error: 0.2212 - val_loss: 0.4367 - val_mean_squared_error: 0.4367 - val_rmse: 0.6608 - val_mean_absolute_error: 0.5034 - lr: 1.9531e-05
Epoch 372/1000
7/7 [==============================] - ETA: 0s - loss: 0.0719 - mean_squared_error: 0.0719 - rmse: 0.2681 - mean_absolute_error: 0.2147
Epoch 00372: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 356ms/step - loss: 0.0719 - mean_squared_error: 0.0719 - rmse: 0.2681 - mean_absolute_error: 0.2147 - val_loss: 0.4378 - val_mean_squared_error: 0.4378 - val_rmse: 0.6617 - val_mean_absolute_error: 0.5038 - lr: 1.9531e-05
Epoch 373/1000
7/7 [==============================] - ETA: 0s - loss

Epoch 389/1000
7/7 [==============================] - ETA: 0s - loss: 0.0911 - mean_squared_error: 0.0911 - rmse: 0.3018 - mean_absolute_error: 0.2472
Epoch 00389: val_loss did not improve from 0.42734
7/7 [==============================] - 4s 500ms/step - loss: 0.0911 - mean_squared_error: 0.0911 - rmse: 0.3018 - mean_absolute_error: 0.2472 - val_loss: 0.4315 - val_mean_squared_error: 0.4315 - val_rmse: 0.6569 - val_mean_absolute_error: 0.5025 - lr: 9.7656e-06
Epoch 390/1000
7/7 [==============================] - ETA: 0s - loss: 0.0868 - mean_squared_error: 0.0868 - rmse: 0.2946 - mean_absolute_error: 0.2444
Epoch 00390: val_loss did not improve from 0.42734
7/7 [==============================] - 3s 494ms/step - loss: 0.0868 - mean_squared_error: 0.0868 - rmse: 0.2946 - mean_absolute_error: 0.2444 - val_loss: 0.4316 - val_mean_squared_error: 0.4316 - val_rmse: 0.6570 - val_mean_absolute_error: 0.5025 - lr: 9.7656e-06
Epoch 391/1000
7/7 [==============================] - ETA: 0s - loss

Epoch 407/1000
7/7 [==============================] - ETA: 0s - loss: 0.0598 - mean_squared_error: 0.0598 - rmse: 0.2446 - mean_absolute_error: 0.1973
Epoch 00407: val_loss did not improve from 0.42734

Epoch 00407: ReduceLROnPlateau reducing learning rate to 4.882812390860636e-06.
5/5 [==============================] - 0s 31ms/step - loss: 14.4578 - mean_squared_error: 14.4578 - rmse: 3.8023 - mean_absolute_error: 2.8097


In [13]:
df_single = (single0.merge(single3, how='outer')).merge(single4, how='outer')

In [14]:
df_single

,Model,Train_site,Test_site,Metric,Value
0,cnn_resnet,0,0,MSE,3.337066
1,cnn_resnet,0,0,RMSE,1.826764
2,cnn_resnet,0,0,MAE,1.433299
3,cnn_resnet,0,0,coef,0.750943
4,cnn_resnet,0,3,MSE,32.367920
5,cnn_resnet,0,3,RMSE,5.689281
6,cnn_resnet,0,3,MAE,3.607088
7,cnn_resnet,0,3,coef,0.033309
8,cnn_resnet,0,4,MSE,11.005393
9,cnn_resnet,0,4,RMSE,3.317438


In [15]:
within_site_mae = df_single[(df_single['Metric'] == 'MAE')][(df_single['Test_site'] == df_single['Train_site'])]

/var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/ipykernel_5576/4111729130.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  within_site_mae = df_single[(df_single['Metric'] == 'MAE')][(df_single['Test_site'] == df_single['Train_site'])]


In [16]:
within_site_mae

,Model,Train_site,Test_site,Metric,Value
2,cnn_resnet,0,0,MAE,1.433299
14,cnn_resnet,3,3,MAE,1.686878
26,cnn_resnet,4,4,MAE,2.032226


In [17]:
cross_site_mae = df_single[(df_single['Metric'] == 'MAE')][(df_single['Test_site'] != df_single['Train_site'])]

/var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/ipykernel_5576/2643963919.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cross_site_mae = df_single[(df_single['Metric'] == 'MAE')][(df_single['Test_site'] != df_single['Train_site'])]


In [18]:
cross_site_mae

,Model,Train_site,Test_site,Metric,Value
6,cnn_resnet,0,3,MAE,3.607088
10,cnn_resnet,0,4,MAE,2.444590
18,cnn_resnet,3,0,MAE,3.084793
22,cnn_resnet,3,4,MAE,2.625219
30,cnn_resnet,4,0,MAE,2.721054
34,cnn_resnet,4,3,MAE,2.809716


#### double-cross-site

In [19]:
double_34, history_double_34 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 3, 4, 0, X, y)
double_04, history_double_04 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 4, 3, X, y)
double_03, history_double_03 = double_cross_site(cnn_resnet, 'cnn_resnet', 0.01, 0, 3, 4, X, y)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d_33 (Conv1D)             (None, 100, 64)      36928       ['input_4[0][0]']                
                                                                                                  
 batch_normalization_36 (BatchN  (None, 100, 64)     256         ['conv1d_33[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_36 (Activation)     (None, 100, 64)      0           ['batch_normalization_36[0]

13/13 [==============================] - ETA: 0s - loss: 59.5811 - mean_squared_error: 59.5811 - rmse: 7.7189 - mean_absolute_error: 6.2017
Epoch 00001: val_loss improved from inf to 343947488.00000, saving model to /var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/tmpwa7i7jze.h5
13/13 [==============================] - 15s 596ms/step - loss: 59.5811 - mean_squared_error: 59.5811 - rmse: 7.7189 - mean_absolute_error: 6.2017 - val_loss: 343947488.0000 - val_mean_squared_error: 343947488.0000 - val_rmse: 18545.8223 - val_mean_absolute_error: 18443.7578 - lr: 0.0100
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 9.0257 - mean_squared_error: 9.0257 - rmse: 3.0043 - mean_absolute_error: 2.4306
Epoch 00002: val_loss improved from 343947488.00000 to 3256439.50000, saving model to /var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/tmpwa7i7jze.h5
13/13 [==============================] - 7s 548ms/step - loss: 9.0257 - mean_squared_error: 9.0257 - rmse: 3.0043 - mean_absolute_

Epoch 17/1000
13/13 [==============================] - ETA: 0s - loss: 1.4228 - mean_squared_error: 1.4228 - rmse: 1.1928 - mean_absolute_error: 0.9248
Epoch 00017: val_loss did not improve from 20.40281
13/13 [==============================] - 7s 514ms/step - loss: 1.4228 - mean_squared_error: 1.4228 - rmse: 1.1928 - mean_absolute_error: 0.9248 - val_loss: 29.3281 - val_mean_squared_error: 29.3281 - val_rmse: 5.4155 - val_mean_absolute_error: 4.5476 - lr: 0.0100
Epoch 18/1000
13/13 [==============================] - ETA: 0s - loss: 1.2933 - mean_squared_error: 1.2933 - rmse: 1.1372 - mean_absolute_error: 0.8721
Epoch 00018: val_loss did not improve from 20.40281
13/13 [==============================] - 7s 540ms/step - loss: 1.2933 - mean_squared_error: 1.2933 - rmse: 1.1372 - mean_absolute_error: 0.8721 - val_loss: 25.2560 - val_mean_squared_error: 25.2560 - val_rmse: 5.0255 - val_mean_absolute_error: 4.3090 - lr: 0.0100
Epoch 19/1000
13/13 [==============================] - ETA: 0s -

Epoch 34/1000
13/13 [==============================] - ETA: 0s - loss: 0.5449 - mean_squared_error: 0.5449 - rmse: 0.7382 - mean_absolute_error: 0.5767
Epoch 00034: val_loss improved from 7.93617 to 5.80054, saving model to /var/folders/tm/dw_3jxc93lq1gxwjq5dqhwph0000gn/T/tmpwa7i7jze.h5
13/13 [==============================] - 7s 519ms/step - loss: 0.5449 - mean_squared_error: 0.5449 - rmse: 0.7382 - mean_absolute_error: 0.5767 - val_loss: 5.8005 - val_mean_squared_error: 5.8005 - val_rmse: 2.4084 - val_mean_absolute_error: 1.9075 - lr: 0.0100
Epoch 35/1000
13/13 [==============================] - ETA: 0s - loss: 0.4838 - mean_squared_error: 0.4838 - rmse: 0.6956 - mean_absolute_error: 0.5406
Epoch 00035: val_loss did not improve from 5.80054
13/13 [==============================] - 7s 519ms/step - loss: 0.4838 - mean_squared_error: 0.4838 - rmse: 0.6956 - mean_absolute_error: 0.5406 - val_loss: 5.8947 - val_mean_squared_error: 5.8947 - val_rmse: 2.4279 - val_mean_absolute_error: 1.880

13/13 [==============================] - ETA: 0s - loss: 0.4683 - mean_squared_error: 0.4683 - rmse: 0.6843 - mean_absolute_error: 0.5393
Epoch 00051: val_loss did not improve from 3.18837
13/13 [==============================] - 7s 520ms/step - loss: 0.4683 - mean_squared_error: 0.4683 - rmse: 0.6843 - mean_absolute_error: 0.5393 - val_loss: 10.8427 - val_mean_squared_error: 10.8427 - val_rmse: 3.2928 - val_mean_absolute_error: 2.8404 - lr: 0.0100
Epoch 52/1000
13/13 [==============================] - ETA: 0s - loss: 0.4129 - mean_squared_error: 0.4129 - rmse: 0.6426 - mean_absolute_error: 0.5009
Epoch 00052: val_loss did not improve from 3.18837
13/13 [==============================] - 7s 503ms/step - loss: 0.4129 - mean_squared_error: 0.4129 - rmse: 0.6426 - mean_absolute_error: 0.5009 - val_loss: 14.4168 - val_mean_squared_error: 14.4168 - val_rmse: 3.7969 - val_mean_absolute_error: 3.2257 - lr: 0.0100
Epoch 53/1000
13/13 [==============================] - ETA: 0s - loss: 0.4600 - 

...

13/13 [==============================] - ETA: 0s - loss: 0.0930 - mean_squared_error: 0.0930 - rmse: 0.3050 - mean_absolute_error: 0.2345
Epoch 00329: val_loss did not improve from 0.61746
13/13 [==============================] - 4s 287ms/step - loss: 0.0930 - mean_squared_error: 0.0930 - rmse: 0.3050 - mean_absolute_error: 0.2345 - val_loss: 0.6288 - val_mean_squared_error: 0.6288 - val_rmse: 0.7929 - val_mean_absolute_error: 0.5861 - lr: 3.9062e-05
Epoch 330/1000
13/13 [==============================] - ETA: 0s - loss: 0.1541 - mean_squared_error: 0.1541 - rmse: 0.3926 - mean_absolute_error: 0.3011
Epoch 00330: val_loss did not improve from 0.61746
13/13 [==============================] - 4s 288ms/step - loss: 0.1541 - mean_squared_error: 0.1541 - rmse: 0.3926 - mean_absolute_error: 0.3011 - val_loss: 0.6257 - val_mean_squared_error: 0.6257 - val_rmse: 0.7910 - val_mean_absolute_error: 0.5858 - lr: 3.9062e-05
Epoch 331/1000
13/13 [==============================] - ETA: 0s - loss: 0.0

13/13 [==============================] - ETA: 0s - loss: 0.1256 - mean_squared_error: 0.1256 - rmse: 0.3544 - mean_absolute_error: 0.2644
Epoch 00346: val_loss did not improve from 0.61746
13/13 [==============================] - 4s 279ms/step - loss: 0.1256 - mean_squared_error: 0.1256 - rmse: 0.3544 - mean_absolute_error: 0.2644 - val_loss: 0.6284 - val_mean_squared_error: 0.6284 - val_rmse: 0.7927 - val_mean_absolute_error: 0.5885 - lr: 1.9531e-05
Epoch 347/1000
13/13 [==============================] - ETA: 0s - loss: 0.0501 - mean_squared_error: 0.0501 - rmse: 0.2238 - mean_absolute_error: 0.1763
Epoch 00347: val_loss did not improve from 0.61746
13/13 [==============================] - 3s 264ms/step - loss: 0.0501 - mean_squared_error: 0.0501 - rmse: 0.2238 - mean_absolute_error: 0.1763 - val_loss: 0.6307 - val_mean_squared_error: 0.6307 - val_rmse: 0.7942 - val_mean_absolute_error: 0.5893 - lr: 1.9531e-05
Epoch 348/1000
13/13 [==============================] - ETA: 0s - loss: 0.0

13/13 [==============================] - ETA: 0s - loss: 0.1648 - mean_squared_error: 0.1648 - rmse: 0.4060 - mean_absolute_error: 0.3206
Epoch 00363: val_loss did not improve from 0.61746
13/13 [==============================] - 3s 270ms/step - loss: 0.1648 - mean_squared_error: 0.1648 - rmse: 0.4060 - mean_absolute_error: 0.3206 - val_loss: 0.6242 - val_mean_squared_error: 0.6242 - val_rmse: 0.7901 - val_mean_absolute_error: 0.5851 - lr: 9.7656e-06
Epoch 364/1000
13/13 [==============================] - ETA: 0s - loss: 0.0955 - mean_squared_error: 0.0955 - rmse: 0.3091 - mean_absolute_error: 0.2338
Epoch 00364: val_loss did not improve from 0.61746
13/13 [==============================] - 3s 267ms/step - loss: 0.0955 - mean_squared_error: 0.0955 - rmse: 0.3091 - mean_absolute_error: 0.2338 - val_loss: 0.6243 - val_mean_squared_error: 0.6243 - val_rmse: 0.7902 - val_mean_absolute_error: 0.5858 - lr: 9.7656e-06
Epoch 365/1000
13/13 [==============================] - ETA: 0s - loss: 0.1

13/13 [==============================] - ETA: 0s - loss: 0.0972 - mean_squared_error: 0.0972 - rmse: 0.3118 - mean_absolute_error: 0.2423
Epoch 00380: val_loss did not improve from 0.61746
13/13 [==============================] - 4s 295ms/step - loss: 0.0972 - mean_squared_error: 0.0972 - rmse: 0.3118 - mean_absolute_error: 0.2423 - val_loss: 0.6258 - val_mean_squared_error: 0.6258 - val_rmse: 0.7911 - val_mean_absolute_error: 0.5872 - lr: 4.8828e-06
Epoch 381/1000
13/13 [==============================] - ETA: 0s - loss: 0.0937 - mean_squared_error: 0.0937 - rmse: 0.3061 - mean_absolute_error: 0.2425
Epoch 00381: val_loss did not improve from 0.61746
13/13 [==============================] - 3s 271ms/step - loss: 0.0937 - mean_squared_error: 0.0937 - rmse: 0.3061 - mean_absolute_error: 0.2425 - val_loss: 0.6244 - val_mean_squared_error: 0.6244 - val_rmse: 0.7902 - val_mean_absolute_error: 0.5869 - lr: 4.8828e-06
Epoch 382/1000
13/13 [==============================] - ETA: 0s - loss: 0.1

2/2 [==============================] - 0s 19ms/step - loss: 9.5379 - mean_squared_error: 9.5379 - rmse: 3.0884 - mean_absolute_error: 2.5703


In [20]:
df_double = (double_34.merge(double_04, how='outer')).merge(double_03, how='outer')

In [21]:
double_mae = df_double[(df_double['Metric'] == 'MAE')]

In [22]:
double_mae

,Model,Train_site,Test_site,Metric,Value
2,cnn_resnet,"3, 4",3,MAE,1.906884
6,cnn_resnet,"3, 4",4,MAE,1.975941
10,cnn_resnet,"3, 4",0,MAE,2.687482
14,cnn_resnet,"0, 4",0,MAE,1.615712
18,cnn_resnet,"0, 4",4,MAE,2.030313
22,cnn_resnet,"0, 4",3,MAE,2.668012
26,cnn_resnet,"0, 3",0,MAE,1.960988
30,cnn_resnet,"0, 3",3,MAE,1.864035
34,cnn_resnet,"0, 3",4,MAE,2.570347


In [23]:
cross_site_mae

,Model,Train_site,Test_site,Metric,Value
6,cnn_resnet,0,3,MAE,3.607088
10,cnn_resnet,0,4,MAE,2.444590
18,cnn_resnet,3,0,MAE,3.084793
22,cnn_resnet,3,4,MAE,2.625219
30,cnn_resnet,4,0,MAE,2.721054
34,cnn_resnet,4,3,MAE,2.809716


In [24]:
within_site_mae

,Model,Train_site,Test_site,Metric,Value
2,cnn_resnet,0,0,MAE,1.433299
14,cnn_resnet,3,3,MAE,1.686878
26,cnn_resnet,4,4,MAE,2.032226


In [25]:
for site in [0, 3, 4]: 
    within = np.mean(within_site_mae[within_site_mae["Test_site"] == site]["Value"])
    across = np.mean(cross_site_mae[cross_site_mae["Test_site"] == site]["Value"])
    cost = np.mean(across - within)
    for x in double_mae[double_mae["Test_site"] == site].iterrows():
        if str(site) not in x[1]["Train_site"]:
            across_double = np.mean(x[1]["Value"])
            cost_double = np.mean(across_double - within)
            
    print("site:", site)
    print("within: ", within)
    print("across: ", across)
    print("cost: ", cost)
    print("across_double: ", across_double)
    print("cost_double: ", cost_double)
    print("################")

site: 0
within:  1.4332993030548096
across:  2.902923583984375
cost:  1.4696242809295654
across_double:  2.6874821186065674
cost_double:  1.2541828155517578
################
site: 3
within:  1.686877727508545
across:  3.2084020376205444
cost:  1.5215243101119995
across_double:  2.6680123805999756
cost_double:  0.9811346530914307
################
site: 4
within:  2.0322256088256836
across:  2.5349044799804688
cost:  0.5026788711547852
across_double:  2.5703465938568115
cost_double:  0.5381209850311279
################
